# 1、实验1

这里是希望通过notebook的内存记忆功能，将一些需要跑长时间才能够保存到的中间变量保存下来，方便后续使用
目前思路是：

    1、将TradingInfo表中所有股票的开始时间和结束时间获取到
    2、将该时间更新到StockInfo表中；
    3、上述两步为预处理，预处理完成后， 每次更新股票交易信息步骤如下：
        3.1 更新StockInfo表，（start_time和end_time留空）
        3.2 获取StockInfo表中全部的code，start_time，end_time信息
        3.3 如果start_time or end_time为空，那么start_time选20050101，end_time选今天，去获取交易信息，并写入数据库
        3.4 如果start_time or end_tiem不为空，那么start_time 为end_time+1,end_time为今天，去获取交易信息，并写入数据库

In [3]:
import os
import sys

mod_path = os.path.abspath('./../dataUpdate/')
print(mod_path)
sys.path.append('/Users/roland/vscode_program/backtrader_learning/dataUpdate')


/Users/roland/vscode_program/dataUpdate


In [31]:
from sqlalchemy.orm import Session
from dataUpdate import engine
from dataUpdate.model.basicinfo import StockBasic
from dataUpdate.model.tradingInfo import TradingInfo
from dataUpdate.service import updateService
import sqlalchemy
from sqlalchemy import func
import time
import pandas as pd 
from tqdm import tqdm


In [10]:
sess = Session(engine)

In [13]:
# 1、获取全部股票代码
sql = sess.query(StockBasic.code)
stock_codes = pd.read_sql(sql.statement, sql.session.bind)


In [17]:
t0 = time.time()
sql2 = sess.query(TradingInfo.code, func.min(TradingInfo.time),func.max(TradingInfo.time)). \
    group_by(TradingInfo.code)
stock_last_date = pd.read_sql(sql2.statement,sql2.session.bind)
t1 = time.time()
print(t1-t0)



1264.2110788822174


In [18]:
stock_last_date.head()

,code,min_1,max_1
0,sh.600000,2005-01-04 10:00:00,2021-09-09 15:00:00
1,sh.600004,2005-01-04 10:00:00,2021-09-09 15:00:00
2,sh.600006,2005-01-04 10:00:00,2021-09-09 15:00:00
3,sh.600007,2005-01-04 10:00:00,2021-09-09 15:00:00
4,sh.600008,2005-01-04 10:00:00,2021-09-09 15:00:00


In [19]:
all_stocks = sess.query(StockBasic).all()


In [32]:
for stk in tqdm(all_stocks):
    # print(stk.code)
    try:
        temp_res  = stock_last_date[stock_last_date['code']==stk.code]
        start,end = temp_res['min_1'].item(),temp_res['max_1'].item()
        stk.start_time=start
        stk.end_time = end
    except Exception as e:
        print(e)
        print(stk.code)
        print(temp_res)
        print("*"*10)
sess.commit()


 17%|█▋        | 756/4476 [00:00<00:03, 1058.80it/s]


ValueError: can only convert an array of size 1 to a Python scalar

# 2、实验2 

    done 1、验证尝akshare上的业绩快报信息 
    done 2、akshare的股票code和保存的既有code之间需要转换
        2.1 6开头 上交所，0开头，深交所
        2.2 002开头，中小板，也是深交所
        2.3 3开头 创业板，也是深交所
        2.4 688开头，科创板，上交所
    done 3、收到的表格名称都是汉字，需要有中英对照表
        /Users/roland/vscode_program/backtrader_learning/dataUpdate/util/namedict.json
    4、科学计数法的存储办法:
        好像也可以不用特别处理
    5、完成数据的第一次写入了，但感觉比较蠢，在思考有没有比较好的办法来处理

In [1]:
import akshare as ak

In [2]:
stock_em_yjbb_df = ak.stock_em_yjbb(date="20100331")


In [3]:
type(stock_em_yjbb_df)

pandas.core.frame.DataFrame

In [4]:
stock_em_yjbb_df.head()

,序号,股票代码,股票简称,每股收益,营业收入-营业收入,营业收入-同比增长,营业收入-季度环比增长,净利润-净利润,净利润-同比增长,净利润-季度环比增长,每股净资产,净资产收益率,每股经营现金流量,销售毛利率,所处行业,最新公告日期
0,1,000617,中油资本,-0.0400,3.046625e+08,10.749529,-47.6992,-10572685.25,-134.422353,-231.3498,2.870065,-1.27,-0.100000,3.111316,多元金融,2012-09-14 00:00:00
1,2,000757,浩物股份,-0.0800,1.319204e+08,101.189055,16.3653,-25948235.48,16.468424,-146.0735,-2.952115,0.00,-0.042300,12.714494,汽车行业,2012-09-13 00:00:00
2,3,301041,金百泽,NaN,7.030412e+07,NaN,NaN,4112270.94,NaN,NaN,3.743231,NaN,-0.169109,15.906955,电子元件,2012-05-22 00:00:00
3,4,600580,卧龙电驱,0.1246,6.553249e+08,34.890407,7.8431,46186436.17,42.922703,-10.3024,3.430000,3.62,-0.160000,20.376400,输配电气,2012-04-07 00:00:00
4,5,002602,世纪华通,0.3100,1.972297e+08,NaN,NaN,39728737.60,NaN,NaN,NaN,0.00,-0.030000,34.455695,电子信息,2011-07-27 00:00:00


In [5]:
stock_em_yjbb_df.shape

(2319, 16)

In [12]:
stock_em_yjbb_df2 = ak.stock_em_yjbb(date="20200331")



In [13]:
stock_em_yjbb_df2.rename(columns={
  "股票代码":"code",
  "股票简称":"name",
  "每股收益":"earning_per_share",
  "营业收入-营业收入":"revenue",
  "营业收入-同比增长":"revenue_yoy_rise",
  "营业收入-季度环比增长":"revenue_sos_rise",
  "净利润-净利润":"net_profit",
  "净利润-同比增长":"net_profit_yoy_rise",
  "净利润-季度环比增长":"net_profit_sos_rise",
  "每股净资产":"net_asset_value_per_share",
  "净资产收益率":"return_on_equity",
  "每股经营现金流量":"operating_cash_flow_per_share",
  "销售毛利率":"gross_profit_ratio",
  "所处行业":"industry",
  "最新公告日期":"date"
},inplace=True)

In [14]:
stock_em_yjbb_df2.columns

Index(['序号', 'code', 'name', 'earning_per_share', 'revenue',
       'revenue_yoy_rise', 'revenue_sos_rise', 'net_profit',
       'net_profit_yoy_rise', 'net_profit_sos_rise',
       'net_asset_value_per_share', 'return_on_equity',
       'operating_cash_flow_per_share', 'gross_profit_ratio', 'industry',
       'date'],
      dtype='object')

In [15]:
stock_em_yjbb_df2.head()

,序号,code,name,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,net_asset_value_per_share,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,industry,date
0,1,430564,天润科技,-0.0700,1.126686e+07,NaN,NaN,-2.911283e+06,NaN,NaN,NaN,-3.58,NaN,23.325943,None,2021-09-08 00:00:00
1,2,872967,蓉中电气,0.0300,2.505979e+07,NaN,-71.0354,1.710819e+06,NaN,-64.2536,NaN,2.02,NaN,22.783039,None,2021-09-07 00:00:00
2,3,000752,*ST西发,-0.0900,4.647864e+07,-34.759410,-45.5438,-2.290606e+07,4.77,91.7905,0.047763,-95.24,-0.052915,17.168526,酿酒行业,2021-09-01 00:00:00
3,4,603766,隆鑫通用,0.0517,1.555182e+09,-29.446706,-48.0266,1.061947e+08,-47.41,1205.4320,3.426357,1.52,-0.122235,16.278215,机械行业,2021-08-31 00:00:00
4,5,400059,天珑5,0.0400,1.384542e+09,15.000000,-53.2368,6.872490e+07,22.81,364.4805,1.471624,2.51,0.171434,16.427678,None,2021-08-31 00:00:00


In [21]:
type(stock_em_yjbb_df2.iloc[0]['revenue'].item())

float

In [22]:
stock_em_yjbb_df2.iloc[0]['revenue'].item()

11266863.63

In [23]:
import datetime

a = datetime.datetime.now()
a

datetime.datetime(2021, 9, 13, 14, 11, 26, 113499)

In [25]:
a.year

2021

0 13
0 14
0 15
1 13
1 14
1 15
2 13
2 14
2 15
3 13
3 14
3 15
4 13
4 14
4 15


In [1]:
from sqlalchemy.orm import Session
from dataUpdate import engine
import pandas as pd


In [2]:
from dataUpdate.model.basicinfo import StockEarningReport
sess = Session(engine)


In [3]:
sql1 = sess.query(StockEarningReport).filter(StockEarningReport.code=='601919')
res_df = pd.read_sql(sql1.statement,sql1.session.bind)

In [4]:
res_df.sort_values(by='time',inplace=True)

In [5]:
99+38+11+2.9

150.9

In [6]:
res_df.tail(10)

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,time
36,544,601919,0.06,3.507510e+10,59.989552,-9.3654,6.873561e+08,280.021537,87.0660,2.62,0.333814,11.176304,2019-03-31
37,1063,601919,0.10,7.210116e+10,59.957480,5.5623,1.237231e+09,2932.759557,-20.0014,4.41,0.772202,10.913706,2019-06-30
38,366,601919,0.18,1.116173e+11,35.903076,6.7252,2.116274e+09,145.340631,59.8621,7.31,1.223705,10.927168,2019-09-30
39,10131,601919,0.56,1.510567e+11,25.016363,-0.1942,6.764105e+09,449.920000,428.7379,21.57,1.729460,10.742339,2019-12-31
40,533,601919,0.02,3.610890e+10,2.947390,-8.4446,2.915195e+08,-57.590000,-93.7278,0.82,0.125133,8.907631,2020-03-31
41,446,601919,0.07,7.405293e+10,2.706986,5.0822,1.137164e+09,-8.090000,190.0816,3.15,0.932963,9.349992,2020-06-30
42,423,601919,0.31,1.177087e+11,5.457442,15.0531,3.860160e+09,82.400000,222.0024,10.38,1.854359,11.509933,2020-09-30
43,10130,601919,0.81,1.712588e+11,13.373888,22.6643,9.927098e+09,46.760000,122.8038,25.05,3.673106,14.208792,2020-12-31
44,543,601919,1.26,6.484281e+10,79.575722,21.0881,1.545233e+10,5200.620000,154.6973,29.89,2.200588,33.208635,2021-03-31
45,449,601919,2.33,1.392643e+11,88.060550,14.7722,3.709784e+10,3162.310000,40.0793,59.65,5.187370,38.240002,2021-06-30


In [8]:
a = pd.DataFrame({"a":[1,2,3,5]})
a

,a
0,1
1,2
2,3
3,5


In [9]:
a['a'] = 10

In [10]:
a

,a
0,10
1,10
2,10
3,10


In [12]:
a['b']=1
a

,a,b
0,10,1
1,10,1
2,10,1
3,10,1


In [14]:
a['b'] = list(range(1,len(a)+1))
a

,a,b
0,10,1
1,10,2
2,10,3
3,10,4
